Adding solar system object (SSO) metric capabilities to [MAF](https://confluence.lsstcorp.org/display/SIM/MAF+documentation) moves far enough beyond the current concepts of slicers + metrics in MAF that it seems worthwhile to prototype the functionality required, and then see how we should map this back into classes for MAF.

The general concept in MAF (for stationary objects) is: 
* read the pointing history from the opsim sqlite database
* determine how to subdivide the pointing history appropriately for a desired metric measurement + use the appropriate `slicer` to do so
* iterate over the `slicer` and apply a `metric` (python class which makes a measurement on a given data slice) at each slice|
* consolidate the metric results at each slicepoint, as appropriate given the slicer (i.e. make a skymap, etc.)

For moving objects, we have additional steps and instead of the metric being applied per "subdivision of the pointing history" (per RA/Dec pointing, for example), we probably want to think of the metric being applied "per SSO". 
So the steps could be:
* read the orbital parameters and Hmag for each SSO
* read the pointing history from the opsim sqlite database
* determine when each individual object would be detected and what its magnitude would be in each detection (potentially, only record observations where the detection is > 5sigma??)
* apply metric based on observations
*  .. additional wrinkles: because this is slow (per object), we are also interested in (a) saving the pointing history for all objects and (b) allowing each object to be cloned to a variety of different H mags (equivalent to cloning over the size distribution, so that each single orbital parameter set represents a range of different objects). 

---

In [37]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas
import pyoorb as oo

Get the pointing history information from the opsim sqlite output (for now, just a limited subset of the nights)

In [5]:
from lsst.sims.maf.db import OpsimDatabase

In [6]:
dbAddress = 'sqlite:///enigma_1189_sqlite.db'
oo = OpsimDatabase(dbAddress)

In [10]:
dbcols = ['expMJD', 'fieldRA', 'fieldDec', 'filter', 'finSeeing', 'fiveSigmaDepth']
sqlconstraint = 'night<60'  
simdata = oo.fetchMetricData(dbcols, sqlconstraint)
print simdata.dtype.names
print simdata[0]

('obsHistID', 'expMJD', 'fieldRA', 'fieldDec', 'filter', 'finSeeing', 'fiveSigmaDepth')
(1, 49353.032079, 1.676483, -1.082473, 'y', 0.954372, 21.084291)


---
Read the moving object orbital information. For now, we'll use pandas to read it in, but this should be done in MAF later, similarly to how the opsim pointings are read.

In [54]:
orbits = pandas.read_table('pha20141031.des', sep=' ')

In [55]:
orbits[0:3]

,!!ObjID,FORMAT,q,e,i,Omega/node,omega/argperi,t_p,magHv,t_0,INDEX,N_PAR,MOID,COMPCODE
0,0,COM,0.186523,0.826961,22.827817,88.024464,31.358208,49090.767296,20,49353.16,1,6,10,PYOORB
1,1,COM,0.827770,0.335431,13.337659,337.219357,276.868840,48990.932835,20,49353.16,1,6,10,PYOORB
2,2,COM,0.646745,0.560063,6.353077,35.696938,285.875200,49239.645606,20,49353.16,1,6,10,PYOORB


In [56]:
orbits = orbits.to_records()
print len(orbits), orbits.dtype.names, orbits[0]

1510 ('index', '!!ObjID', 'FORMAT', 'q', 'e', 'i', 'Omega/node', 'omega/argperi', 't_p', 'magHv', 't_0', 'INDEX', 'N_PAR', 'MOID', 'COMPCODE') (0, 0, 'COM', 0.1865233429220149, 0.82696095, 22.827817, 88.02446399999998, 31.358208, 49090.767295844285, 20.0, 49353.16, 1, 6, 10, 'PYOORB')


It's too expensive to calculate ephemerides for every object at the time of every opsim pointing. Let's start with calculating the positions every 2 hours for PHAs.  Because pyoorb is a python interface to a Fortran program (OpenOrb) we have to do some awkwardness with array-rearranging.

In [78]:
timestep = 2.0 / 24.0  # in days
times = np.arange(simdata['expMJD'][0], simdata['expMJD'][-1] + timestep/2.0, timestep)
ephTimescale = 4*np.ones(len(times))  # 1= MJD_UTC, 2=UT1, 3=TT, 4=TAI (timescale for the ephemerides)
ephTimes = np.array(zip(times, ephTimescale), dtype='double', order='F')

Set up pyoorb (reads in JPL ephemerides for major planets, etc.). We could potentially use command-line OpenOrb, and spawn from python .. but since it wants file input/output rather than stdin/out this seems problematic.

In [79]:
oo.pyoorb.oorb_init(ephemeris_fname="")

1

Python oorb element format: 
        # Translate orbital elements into array that pyoorb will like.
        # PyOrb wants ::
        # 0: orbitId  (cannot be a string)
        # 1 - 6: orbital elements, using radians for angles
        # 7: element type code, where 2 = cometary - means timescale is TT, too
        # 8: epoch
        # 9: timescale for the epoch; 1= MJD_UTC, 2=UT1, 3=TT, 4=TAI
        # 10: magHv
        # 11: G
so we have to do a little translating from the orbits DataFrame to the elements we want in this array. 

Also, the ephemeride generation can be more efficiently done for multiple objects at a time, but then we have to figure out how to balance the amount of ephemerides created (for how many objects, for how long) vs. memory available and how we want to access them later. Let's see if the simplest method (iterating over the objects one by one) will be fast enough.

More awkwardness with the ephems and elems arrays:
       # Returned ephems contain a 3-D Fortran array of ephemerides, the axes are:
        #   [objid][time][ephemeris information element]
        # the ephemeris information elements are (in order):
        # distance, ra, dec, mag, ephem mjd, ephem mjd timescale, dradt(sky), ddecdt(sky)
        # per object, per date, 8 elements (array shape is OBJ(s)/DATE(s)/VALUES)
        # Note that ra/dec, dradt, etc. are all in DEGREES.

In [100]:
for sso in orbits[1:2]:
    elems = [sso['!!ObjID'], sso['q'], sso['e'], np.radians(sso['i']), np.radians(sso['Omega/node']), 
             np.radians(sso['omega/argperi']), sso['t_p'],  2, sso['t_0'], 3, sso['magHv'], 0.15]
    elems = np.column_stack(elems)
    ephems, err = oo.pyoorb.oorb_ephemeris(in_orbits = elems, in_obscode=807, in_date_ephems=ephTimes)
    print ephems[0][0][4]
    print times[0]

49353.032079
49353.032079
